# Crear un TorchPlayer


Recibe el modelo a instanciar como path y juega con el mismo

- Pensar como resolver el problema de que solo samplee las válidas
- Agregarle la opción de monte carlo tree search (opcional) con las opciones de iterationLimit, timeLimit

Si va a agregar MCTS mirar la notebook 007_MCTS.ipnb

In [35]:
from players import GreedyPlayer, RandomPlayer
from multi_env import make_reversi_vec_env, SelfPlayEnv
from boardgame2 import ReversiEnv
from stable_baselines3 import PPO
from stable_baselines3.common.policies import ActorCriticPolicy
import numpy as np

In [26]:
board_shape = 8
n_envs = 10
env = make_reversi_vec_env(
    SelfPlayEnv, n_envs=n_envs,
    env_kwargs={
        'board_shape': board_shape,
        'LocalPlayer': RandomPlayer
    }
)

In [27]:
model = PPO(
    ActorCriticPolicy,
    env,
    verbose=0,
)

In [28]:
class TorchPlayer():
    # 
    def __init__(
        self, model_path=None, 
        player=1, board_shape=None, 
        env=None, flatten_action=False,
        deterministic=True, only_valid=True, 
        mcts=False, iterationLimit=None, timeLimit=None):
        if model_path is None:
            model_path = '/home/rl_final_project/models/Reversi_PPO_8by8_0.99_0.95_0.0_20_6_masked_actions/best_model.zip'
        self.model = PPO.load(model_path)
        self.player = player
        self.flatten_action = flatten_action
        if (env is None) and (board_shape is None):
            print("board_shape and env can't be both None")
        if env is None:
            self.env = self.model.env
        if board_shape is None:
            self.board_shape = env.board_shape
        else:
            self.board_shape = board_shape
    
    def predict(self, board):
        board = board * self.player
        board_reshape = board[np.newaxis,:,:]
        action = self.model.predict(board_reshape)[0]
        if self.flatten_action:
            return action
        else:
            return [action // self.board_shape, action % self.board_shape]

# Arena

Testear el jugador contra los distintos jugadore

In [29]:
def arena_stats(Player_1, Player_2, board_shape, N=500):
    
    env = ReversiEnv(board_shape=board_shape)
    wins_as_first = 0
    wins_as_second = 0
    plays_as_first = 0
    plays_as_second = 0
    total_steps = 0
    player_1 = Player_1(player=1, board_shape=board_shape, flatten_action=False)
    player_2 = Player_2(player=1, board_shape=board_shape, flatten_action=False) # Implementar
    for i in range(N):
        # Aveces empieza un jugador, a veces el otro
        first_player = np.random.choice([-1, 1])
        player_1.player = first_player
        player_2.player = -first_player
        
        plays_as_first = plays_as_first + (first_player == 1)
        plays_as_second = plays_as_second + (first_player == -1)
        
        done = False
        n_steps = 0
        (board, player) = env.reset()
        
        while not done:
            if first_player == player:
                action = player_1.predict(board) # Juega el jugador 1
            else:
                action = player_2.predict(board) # Juega el jugador 2
            (board, player), reward, done, info = env.step(action)
            n_steps = n_steps + 1
        total_steps = total_steps + n_steps
        wins_as_first = wins_as_first + (reward == first_player) * (first_player == 1)
        wins_as_second = wins_as_second + (reward == first_player) * (first_player == -1)
    print(f'Wins as first: {wins_as_first/plays_as_first}')
    print(f'Wins as second: {wins_as_second/plays_as_second}')
    print(f'Plays as first: {plays_as_first}')
    print(f'Plays as second: {plays_as_second}')
    print(f'Avg game duration: {total_steps/N}')

In [31]:
arena_stats(TorchPlayer, RandomPlayer, 8, N=2_000)

Wins as first: 0.6477157360406092
Wins as second: 0.7517241379310344
Plays as first: 985
Plays as second: 1015
Avg game duration: 59.9395


In [32]:
arena_stats(TorchPlayer, GreedyPlayer, 8, N=2_000)

Wins as first: 0.6420626895854399
Wins as second: 0.645895153313551
Plays as first: 989
Plays as second: 1011
Avg game duration: 58.5845
